# Ejercicio OpenStreetMap
Para este ejercicio vas a trabar con Overpass, que es la API que utiliza los datos de OpenStreetMap. Tras obtener datos geográficos mediante la API, tendrás que representarlos con Plotly.

Hay que obtener los siguientes datos de cada ubicación conseguida en OpenStreetMap
* tipo de amenity
* latitud
* longitud
* nombre
* direccion


Tienes en [este enlace](https://wiki.openstreetmap.org/wiki/Key:amenity#Healthcare) la lista de *amenities* que vas a usar en el notebook.

## Instalación
Si no tienes instalado Plotly, abre un terminal y escribe:
```
pip install plotly
```

Importa las librerias que vamos a usar en el Notebook

In [7]:
#!pip install plotly

In [ ]:
import pandas as pd
import numpy as np 


import requests #Objeto para realizar peticiones (APIs), y obtener respuesta... response
import json 

%matplotlib inline

Ataca a la API de OpenStreetmap con el siguiente código de ejemplo

In [9]:

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="ice_cream"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()


<Response [200]>


In [8]:
data

{'version': 0.6,
 'generator': 'Overpass API 0.7.56.8 7d656e78',
 'osm3s': {'timestamp_osm_base': '2020-11-10T19:31:03Z',
  'copyright': 'The data included in this document is from www.openstreetmap.org. The data is made available under ODbL.'},
 'elements': [{'type': 'node',
   'id': 305536401,
   'lat': 40.4230858,
   'lon': -3.6935468,
   'tags': {'amenity': 'restaurant', 'name': 'La Castafiore'}},
  {'type': 'node',
   'id': 324580483,
   'lat': 40.4226068,
   'lon': -3.6932565,
   'tags': {'amenity': 'restaurant',
    'cuisine': 'japanese',
    'name': 'Sake Dining Himawari'}},
  {'type': 'node',
   'id': 3361315181,
   'lat': 40.4233148,
   'lon': -3.6925055,
   'tags': {'addr:city': 'Madrid',
    'addr:housenumber': '12',
    'addr:postcode': '28004',
    'addr:street': 'Calle de Doña Bárbara de Braganza',
    'addr:subdistrict': 'Justicia',
    'amenity': 'restaurant',
    'cuisine': 'regional',
    'name': 'Ainhoa',
    'phone': '+34 913 08 27 26',
    'website': 'http://resta

Con el siguiente código podrás adaptar el JSON que devuelve la API, a un formato DataFrame más amigable

In [ ]:

def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)


Utilizarás esta función para representar los datos en un mapa de Matplotlib

In [5]:

import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()


Como ejemplo, si quiero ver cuántos restaurantes tengo cerca de The Bridge:

In [6]:

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="restaurant"]
  (40.420801, -3.694702,40.423754, -3.688167); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df


<Response [200]>


,tipo,lat,lon,name,address
0,restaurant,40.423086,-3.693547,La Castafiore,NO STREET 9999
1,restaurant,40.422607,-3.693256,Sake Dining Himawari,NO STREET 9999
2,restaurant,40.423315,-3.692506,Ainhoa,Calle de Doña Bárbara de Braganza 12
3,restaurant,40.421960,-3.694270,La Kitchen,NO STREET 9999
4,restaurant,40.421956,-3.693905,En La Gloria,Calle de Prim 9
5,restaurant,40.423695,-3.691651,Café Espejo,NO STREET 9999
6,restaurant,40.421951,-3.693535,Tampu,Calle de Prim 13
7,restaurant,40.422266,-3.694097,Casa Gades,Calle del Conde de Xiquena 4
8,restaurant,40.423151,-3.694062,El Cano de Xiquena,NO STREET 9999
9,restaurant,40.423345,-3.692632,El Gato Persa,Calle de Doña Bárbara de Braganza 10


In [7]:
pinta_mapa(df)

## Ejercicio 1
### Búsqueda de aparcamiento
Para este caso de uso imagina que vas a Madrid de viaje y estás buscando posibles parkings donde dejar el coche, crea un DataFrame con todos los parkings (de OpenStreetMap) de la ciudad de Madrid. Para ello define una cuadrícula con dos coordenadas sobre Madrid.

Representa en un mapa los resultados

In [12]:
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="parking_entrance"]
  (40.396013, -3.746999,40.480011, -3.625910); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()


<Response [200]>


In [13]:
data

te',
    'amenity': 'parking_entrance',
    'maxheight': '1.9',
    'parking': 'underground'}},
  {'type': 'node',
   'id': 5406530355,
   'lat': 40.4031992,
   'lon': -3.6917194,
   'tags': {'access': 'private',
    'amenity': 'parking_entrance',
    'parking': 'underground'}},
  {'type': 'node',
   'id': 5460254007,
   'lat': 40.4717003,
   'lon': -3.7091404,
   'tags': {'access': 'private',
    'amenity': 'parking_entrance',
    'parking': 'underground'}},
  {'type': 'node',
   'id': 5484482336,
   'lat': 40.4001538,
   'lon': -3.7075366,
   'tags': {'access': 'private',
    'amenity': 'parking_entrance',
    'parking': 'underground'}},
  {'type': 'node',
   'id': 5501428617,
   'lat': 40.4297841,
   'lon': -3.7191938,
   'tags': {'amenity': 'parking_entrance'}},
  {'type': 'node',
   'id': 5501428620,
   'lat': 40.429498,
   'lon': -3.7198074,
   'tags': {'amenity': 'parking_entrance'}},
  {'type': 'node',
   'id': 5706130155,
   'lat': 40.4711586,
   'lon': -3.694423,
   'tags': {

In [14]:
df = json_to_df(data)
df

,tipo,lat,lon,name,address
0,parking_entrance,40.469806,-3.641920,NO NAME,NO STREET 9999
1,parking_entrance,40.469150,-3.641935,NO NAME,NO STREET 9999
2,parking_entrance,40.477584,-3.707132,NO NAME,NO STREET 9999
3,parking_entrance,40.478589,-3.702829,NO NAME,NO STREET 9999
4,parking_entrance,40.479067,-3.713584,NO NAME,NO STREET 9999
...,...,...,...,...,...
354,parking_entrance,40.425900,-3.709816,NO NAME,NO STREET 9999
355,parking_entrance,40.426260,-3.709439,NO NAME,NO STREET 9999
356,parking_entrance,40.413905,-3.674708,NO NAME,NO STREET 9999
357,parking_entrance,40.416067,-3.711518,NO NAME,NO STREET 9999


In [18]:
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

In [19]:
pinta_mapa(df)

## Mudanza
Imagina que te quieres mudar al Berrueco, pero antes te gustaría obtener algún indicador de si hay civilización por esa zona. Mira a ver si existe gasolineras y/o colegios por la zona. Define, al igual que en el caso anterior, un rectángulo mediante dos coordenadas. Busca la manera de realizar una única petición a la API.

Representa después en un mapa, por color los colegios y gasolineras

In [21]:

overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
node["amenity"="fuel"]
  (40.851838, -3.627774,40.941457, -3.481755); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df_fuel = json_to_df(data)
df_fuel

<Response [200]>


,tipo,lat,lon,name,address
0,fuel,40.905376,-3.607588,Repsol,NO STREET 9999
1,fuel,40.906677,-3.610172,Shell,CTRA. N-I KM 64.650 9999
2,fuel,40.856872,-3.612894,Repsol,"AVDA. GENERALISIMO,12 9999"
3,fuel,40.875833,-3.602389,Shell,AVENIDA AVDA. DE LA CABRERA S/N JUNTO AL HOSTA...


In [22]:
overpass_query = """
[out:json];
node["amenity"="school"]
  (40.851838, -3.627774,40.941457, -3.481755); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df_school = json_to_df(data)
df_school

<Response [200]>


,tipo,lat,lon,name,address
0,school,40.867720,-3.614696,Escuela infantil de La Cabrera,NO STREET 9999
1,school,40.855890,-3.483306,Patones,NO STREET 9999
2,school,40.872876,-3.606421,La Cabrera,NO STREET 9999
3,school,40.864566,-3.614058,Eq. Aten.Temprana La Cabrera-Lozoyuela,NO STREET 9999
4,school,40.872395,-3.608802,C.A.P. De La Cabrera,NO STREET 9999
5,school,40.863864,-3.615523,Equipo General La Cabrera,NO STREET 9999
6,school,40.872395,-3.608802,C.A.P. De La Cabrera,NO STREET 9999


In [30]:
df = pd.concat([df_fuel, df_school])

In [ ]:

def json_to_df(data):
    elements = data['elements']
    places = {'tipo': [], 'lat': [], 'lon': [], 'name': [], 'address': []}
    
    for i in elements:
        
        tipo = i.get('tags', None).get('amenity', None)
        latitude = i.get('lat', None)
        longitude = i.get('lon', None)
        name = i.get('tags', {}).get('name', "NO NAME")
        street = i.get('tags', {}).get('addr:street', "NO STREET")
        number = i.get('tags', {}).get('addr:housenumber', 9999)

        places['tipo'].append(tipo)
        places['lat'].append(latitude)
        places['lon'].append(longitude)
        places['name'].append(name)
        places['address'].append(street + ' ' + str(number))

            
    return pd.DataFrame(places)

In [27]:
def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
                      
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()

In [31]:
pinta_mapa(df)

## Zonas COVID
Imagina que trabajas en el ayuntamiento de tu barrio/pueblo/ciudad, y por motivos del COVID tienes que identificar:
1. Todas las zonas abiertas de uso común (zonas de barbacoas, fuentes de agua y parques infantiles (kindergarten)), para cerrarlas.
2. Todos los hospitales y clínicas de la zona.

Como no sabes muy bien qué puntos serán, utilizas la API de OpenStreetMap. Representa todo en un único mapa

In [32]:
overpass_query = """
[out:json];
node["amenity"="bbq"]
  (40.43292, -3.829166,40.465285, -3.764212); 
out;
node["amenity"="drinking_water"]
  (40.43292, -3.829166,40.465285, -3.764212); 
out;
node["amenity"="kindergarten"]
  (40.43292, -3.829166,40.465285, -3.764212); 
out;
node["amenity"="hospital"]
  (40.43292, -3.829166,40.465285, -3.764212); 
out;
node["amenity"="clinic"]
  (40.43292, -3.829166,40.465285, -3.764212); 
out;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,drinking_water,40.436855,-3.819278,Fuente de la Poza,NO STREET 9999
1,drinking_water,40.440148,-3.765316,NO NAME,NO STREET 9999
2,drinking_water,40.454084,-3.808987,NO NAME,NO STREET 9999
3,drinking_water,40.447745,-3.789207,NO NAME,NO STREET 9999
4,drinking_water,40.449560,-3.787557,NO NAME,NO STREET 9999
...,...,...,...,...,...
107,clinic,40.445032,-3.784437,RG,NO STREET 9999
108,clinic,40.433924,-3.806175,Dorsia,NO STREET 9999
109,clinic,40.444972,-3.789847,FisioDía,NO STREET 9999
110,clinic,40.438279,-3.794145,Psicología Europa,NO STREET 9999


In [43]:
df[df['tipo']=='bbq'].shape[0]

0

In [42]:
df[df['tipo']=='hospital'].shape[0]

0

In [33]:
pinta_mapa(df)

## Supermercados
Para este ejemplo supondrás que trabajas en una cadena de supermercados en la que se está planteando construir un nuevo local en la provincia de Sevilla. Busca mediante la API de Overpass todos los supermercados de la provincia para poder comprobar en qué nucleos urbanos habría poca competencia.

En este caso hay que buscar por área, no por cuadrícula! Investiga en la documentación/Stackoverflow cómo se haría eso.

In [47]:
# nwr ==> node way relation
overpass_url = "http://overpass-api.de/api/interpreter"
overpass_query = """
[out:json];
area[name="Sevilla"];
nwr[shop=supermarket](area); 
out center;
"""
response = requests.get(overpass_url, 
                        params={'data': overpass_query})
print(response)
data = response.json()
df = json_to_df(data)
df

<Response [200]>


,tipo,lat,lon,name,address
0,None,37.373002,-6.047204,MAS,NO STREET 9999
1,None,37.390016,-5.973769,Maxi Dia,NO STREET 9999
2,None,36.994532,-5.572320,Dia,NO STREET 9999
3,None,37.406364,-5.976342,El Jamón,NO STREET 9999
4,None,37.406508,-5.973342,Valdajimena,Avenida de San Juan de la Salle 9999
...,...,...,...,...,...
452,None,NaN,NaN,Cash Fresh,NO STREET 9999
453,None,NaN,NaN,Covirán,NO STREET 9999
454,None,NaN,NaN,Naturalia,Calle Cultura 34
455,None,NaN,NaN,Aldi,Avenida Doctor Villa 1


In [49]:
df['tipo'] = df['tipo'].fillna("supermarket")
df = df.dropna()
df

,tipo,lat,lon,name,address
0,supermarket,37.373002,-6.047204,MAS,NO STREET 9999
1,supermarket,37.390016,-5.973769,Maxi Dia,NO STREET 9999
2,supermarket,36.994532,-5.572320,Dia,NO STREET 9999
3,supermarket,37.406364,-5.976342,El Jamón,NO STREET 9999
4,supermarket,37.406508,-5.973342,Valdajimena,Avenida de San Juan de la Salle 9999
...,...,...,...,...,...
329,supermarket,37.403192,-5.969346,Lidl,NO STREET 9999
330,supermarket,37.418660,-5.974322,Carrefour,NO STREET 9999
331,supermarket,37.515555,-6.266009,Día,NO STREET 9999
332,supermarket,37.382868,-6.013327,CASH FRESH,Calle San Vicente de Paul 54-56


In [50]:
pinta_mapa(df)

## Etiqueta en el mapa
Busca la manera de modificar la función `pinta_mapa()`, de tal manera que cuando pases el ratón por encima de cada punto, aparezca el nombre de cada nodo. Por ejemplo, que en el último apartado aparezcan los nombres de los supermercados.

In [51]:

import plotly.express as px

mapbox_access_token = "pk.eyJ1IjoiZGFvcnRpIiwiYSI6ImNrZnF6M3FlczA3cDEyem16YTNzZmV4M2EifQ.846iF0sMSAXv0kwkwUTYjg"

def pinta_mapa(df):
    fig = px.scatter_mapbox(df,
                            lat="lat",
                            lon="lon",
                            color = "tipo",
                            hover_name= "name",
                      color_continuous_scale=px.colors.cyclical.IceFire, size_max=15, zoom=11)
    fig.update_layout(
        mapbox=dict(
            accesstoken=mapbox_access_token
        )
    )

    fig.show()


In [52]:
pinta_mapa(df)